# import

In [3]:
import sys
import sys
import os

# Get the current working directory (notebooks do not have __file__)
data_path = os.path.join(os.getcwd(), 'data')
sys.path.append(data_path)


#Use this one if you are running the code from a script not a notebook
# import sys
# import os

# # Add the 'data' directory to the Python path
# data_path = os.path.join(os.path.dirname(os.path.abspath(__file__)), 'data')
# sys.path.append(data_path)

In [4]:
from model.bert import bert_ATE, bert_ABSA

from dataset import dataset_ATM, dataset_ABSA

c:\Users\emink\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from torch.utils.data import DataLoader, ConcatDataset
from transformers import BertTokenizer
import torch
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import time
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from transformers import AdamW

In [6]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#pretrain_model_name = "bert-base-uncased"
pretrain_model_name = "bert-large-uncased"
tokenizer = BertTokenizer.from_pretrained(pretrain_model_name)
lr = 2e-5
model_ATE = bert_ATE(pretrain_model_name).to(DEVICE)
optimizer_ATE = torch.optim.Adam(model_ATE.parameters(), lr=lr)
#model_ABSA = bert_ABSA(pretrain_model_name).to(DEVICE)
model_ABSA = bert_ABSA(pretrain_model_name, DEVICE).to(DEVICE)
#optimizer_ABSA = torch.optim.Adam(model_ABSA.parameters(), lr=lr)
optimizer_ABSA = AdamW(
    model_ABSA.parameters(), 
    lr=lr, 
    weight_decay=1e-4  # or 1e-5
)

c:\Users\emink\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [7]:
def evl_time(t):
    min, sec= divmod(t, 60)
    hr, min = divmod(min, 60)
    return int(hr), int(min), int(sec)

def load_model(model, path):
    model.load_state_dict(torch.load(path), strict=False)
    return model
    
def save_model(model, name):
    torch.save(model.state_dict(), name)

# Acpect Term Extraction

In [6]:

restaurants_train_ds = dataset_ATM(pd.read_csv("data/restaurants_train.csv"), tokenizer)
restaurants_test_ds = dataset_ATM(pd.read_csv("data/restaurants_test.csv"), tokenizer)


In [7]:
# train_ds = ConcatDataset([restaurants_train_ds])
# test_ds = ConcatDataset([restaurants_test_ds])

train_ds =restaurants_train_ds
test_ds = restaurants_test_ds

In [8]:
def create_mini_batch(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors = pad_sequence(ids_tensors, batch_first=True)

    tags_tensors = [s[2] for s in samples]
    tags_tensors = pad_sequence(tags_tensors, batch_first=True)

    pols_tensors = [s[3] for s in samples]
    pols_tensors = pad_sequence(pols_tensors, batch_first=True)
    
    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1)
    
    return ids_tensors, tags_tensors, pols_tensors, masks_tensors

In [10]:
train_loader = DataLoader(train_ds, batch_size=5, collate_fn=create_mini_batch, shuffle = True)
test_loader = DataLoader(test_ds, batch_size=50, collate_fn=create_mini_batch, shuffle = True)

In [10]:
# for batch in train_loader:
#     w,x,y,z = batch
#     print(w)
#     print(w.size())
#     print(x)
#     print(x.size())
#     print(y)
#     print(y.size())
#     print(z)
#     print(z.size())
#     break

In [11]:
def train_model_ATE(loader, epochs):
    all_data = len(loader)
    for epoch in range(epochs):
        finish_data = 0
        losses = []
        current_times = []
        correct_predictions = 0
        
        for data in loader:
            t0 = time.time()
            ids_tensors, tags_tensors, _, masks_tensors = data
            ids_tensors = ids_tensors.to(DEVICE)
            tags_tensors = tags_tensors.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            loss = model_ATE(ids_tensors=ids_tensors, tags_tensors=tags_tensors, masks_tensors=masks_tensors)
            losses.append(loss.item())
            loss.backward()
            optimizer_ATE.step()
            optimizer_ATE.zero_grad()

            finish_data += 1
            current_times.append(round(time.time()-t0,3))
            current = np.mean(current_times)
            hr, min, sec = evl_time(current*(all_data-finish_data) + current*all_data*(epochs-epoch-1))
            print('epoch:', epoch, " batch:", finish_data, "/" , all_data, " loss:", np.mean(losses), " hr:", hr, " min:", min," sec:", sec)         

        save_model(model_ATE, 'bert_ATE.pkl')
        
def test_model_ATE(loader):
    pred = []
    trueth = []
    with torch.no_grad():
        for data in loader:

            ids_tensors, tags_tensors, _, masks_tensors = data
            ids_tensors = ids_tensors.to(DEVICE)
            tags_tensors = tags_tensors.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            outputs = model_ATE(ids_tensors=ids_tensors, tags_tensors=None, masks_tensors=masks_tensors)

            _, predictions = torch.max(outputs, dim=2)

            pred += list([int(j) for i in predictions for j in i ])
            trueth += list([int(j) for i in tags_tensors for j in i ])

    return trueth, pred



In [12]:
%time train_model_ATE(train_loader, 3)  #Tree epoch

epoch: 0  batch: 1 / 721  loss: 1.1760302782058716  hr: 2  min: 47  sec: 3
epoch: 0  batch: 2 / 721  loss: 1.0005813837051392  hr: 2  min: 21  sec: 39
epoch: 0  batch: 3 / 721  loss: 0.8143057525157928  hr: 2  min: 1  sec: 17
epoch: 0  batch: 4 / 721  loss: 0.7612844333052635  hr: 1  min: 50  sec: 0
epoch: 0  batch: 5 / 721  loss: 0.7228415668010711  hr: 1  min: 43  sec: 34
epoch: 0  batch: 6 / 721  loss: 0.6985220064719518  hr: 1  min: 40  sec: 21
epoch: 0  batch: 7 / 721  loss: 0.6895694264343807  hr: 1  min: 37  sec: 2
epoch: 0  batch: 8 / 721  loss: 0.6649655289947987  hr: 1  min: 37  sec: 6
epoch: 0  batch: 9 / 721  loss: 0.6353972984684838  hr: 1  min: 39  sec: 36
epoch: 0  batch: 10 / 721  loss: 0.611596867442131  hr: 1  min: 40  sec: 21
epoch: 0  batch: 11 / 721  loss: 0.5791776857592843  hr: 1  min: 41  sec: 40
epoch: 0  batch: 12 / 721  loss: 0.5523741270105044  hr: 1  min: 41  sec: 29
epoch: 0  batch: 13 / 721  loss: 0.5363121903859652  hr: 1  min: 42  sec: 29
epoch: 0  batc

In [12]:
model_ATE = load_model(model_ATE, 'bert_ATE.pkl')

C:\Users\emink\AppData\Local\Temp\ipykernel_33692\1419488891.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path), strict=False)


In [13]:
def test_sentence_ATE(sentence, tokenizer, model_ATE, device=DEVICE):
    """
    Test the ATE model with a single sentence and print the predictions.

    Args:
        sentence (str): The input sentence.
        tokenizer: The tokenizer used during training.
        model_ATE: The trained ATE model.
        device: The device to run the model on ('cpu' or 'cuda').

    Returns:
        List of predicted tags for the input sentence.
    """
    model_ATE.eval()  # Set the model to evaluation mode

    # Tokenize the input sentence
    tokens = tokenizer.tokenize(sentence)
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids_tensor = torch.tensor([input_ids]).to(device)
    attention_mask = torch.tensor([[1] * len(input_ids)]).to(device)

    # Make predictions
    with torch.no_grad():
        outputs = model_ATE(ids_tensors=input_ids_tensor, tags_tensors=None, masks_tensors=attention_mask)
        _, predictions = torch.max(outputs, dim=2)

    # Convert predictions to tags
    predicted_tags = predictions.squeeze().tolist()
    token_tag_pairs = list(zip(tokens, predicted_tags))

    print("Sentence:", sentence)
    print("Predictions:")
    for token, tag in token_tag_pairs:
        print(f"Token: {token}, Predicted Tag: {tag}")

    return token_tag_pairs


In [14]:
sentence = "The food was amazing but the service was terrible."
token_tag_pairs = test_sentence_ATE(sentence, tokenizer, model_ATE)

sentence = "The pasta was delicious, but the service was slow."
token_tag_pairs= test_sentence_ATE(sentence,tokenizer,model_ATE)

sentence = "The ambiance was fantastic, but the food was overpriced."
token_tag_pairs= test_sentence_ATE(sentence,tokenizer,model_ATE)

sentence = "The waiter was very friendly, and the desserts were outstanding"
token_tag_pairs= test_sentence_ATE(sentence,tokenizer,model_ATE)

Sentence: The food was amazing but the service was terrible.
Predictions:
Token: the, Predicted Tag: 0
Token: food, Predicted Tag: 1
Token: was, Predicted Tag: 0
Token: amazing, Predicted Tag: 0
Token: but, Predicted Tag: 0
Token: the, Predicted Tag: 0
Token: service, Predicted Tag: 1
Token: was, Predicted Tag: 0
Token: terrible, Predicted Tag: 0
Token: ., Predicted Tag: 0
Sentence: The pasta was delicious, but the service was slow.
Predictions:
Token: the, Predicted Tag: 0
Token: pasta, Predicted Tag: 1
Token: was, Predicted Tag: 0
Token: delicious, Predicted Tag: 0
Token: ,, Predicted Tag: 0
Token: but, Predicted Tag: 0
Token: the, Predicted Tag: 0
Token: service, Predicted Tag: 1
Token: was, Predicted Tag: 0
Token: slow, Predicted Tag: 0
Token: ., Predicted Tag: 0
Sentence: The ambiance was fantastic, but the food was overpriced.
Predictions:
Token: the, Predicted Tag: 0
Token: am, Predicted Tag: 1
Token: ##bian, Predicted Tag: 1
Token: ##ce, Predicted Tag: 1
Token: was, Predicted T

In [14]:
%time x, y = test_model_ATE(test_loader)
print(classification_report(x, y, target_names=[str(i) for i in range(3)]))

CPU times: total: 3min 45s
Wall time: 2min 8s
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     64196
           1       0.92      0.81      0.86      4022
           2       0.73      0.85      0.79      2141

    accuracy                           0.98     70359
   macro avg       0.88      0.88      0.88     70359
weighted avg       0.98      0.98      0.98     70359



# Aspect Based Sentiment Analysis

In [15]:

# restaurants_train_ds = dataset_ABSA(pd.read_csv("data/restaurants_train.csv"), tokenizer)
restaurants_train_ds = dataset_ABSA(pd.read_csv("data/ABSA5restaurants_train.csv"), tokenizer)
restaurants_test_ds = dataset_ABSA(pd.read_csv("data/restaurants_test.csv"), tokenizer)
restaurants_val_ds = dataset_ABSA(pd.read_csv("data/V1restaurants_test.csv"), tokenizer)


# restaurants_train_ds = dataset_ABSA(pd.read_csv("data/train_test.csv"), tokenizer)
# restaurants_test_ds = dataset_ABSA(pd.read_csv("data/test_test.csv"), tokenizer)


In [7]:
w,x,y,z = restaurants_train_ds.__getitem__(121)
print(w)
print(len(w))
print(x)
print(len(x))
print(y)
print(len(y))
print(z)

['[cls]', 'i', 'went', 'there', 'in', 'late', 'afternoon', 'for', 'some', 'bite', 'size', 'food', 'and', 'ref', '##les', '##hm', '##ent', 'with', 'my', 'date', '.', '[sep]', 'food']
23
tensor([  100,  1045,  2253,  2045,  1999,  2397,  5027,  2005,  2070,  6805,
         2946,  2833,  1998, 25416,  4244, 14227,  4765,  2007,  2026,  3058,
         1012,   100,  2833])
23
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
23
tensor(1)


In [16]:
def create_mini_batch2(samples):
    # Find the max length in the current batch
    max_len = max([len(s[1]) for s in samples])

    ids_tensors = [torch.cat([s[1], torch.zeros(max_len - len(s[1]), dtype=torch.long)]) for s in samples]
    ids_tensors = torch.stack(ids_tensors)

    segments_tensors = [torch.cat([s[2], torch.zeros(max_len - len(s[2]), dtype=torch.long)]) for s in samples]
    segments_tensors = torch.stack(segments_tensors)

    label_ids = torch.stack([s[3] for s in samples])
    
    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1)

    return ids_tensors, segments_tensors, masks_tensors, label_ids


In [17]:
# train_ds = ConcatDataset([laptops_train_ds, restaurants_train_ds, twitter_train_ds])
# test_ds = ConcatDataset([laptops_test_ds, restaurants_test_ds, twitter_test_ds])

# train_ds = restaurants_train_ds
# test_ds = restaurants_test_ds


# train_loader = DataLoader(train_ds, batch_size=4, collate_fn=create_mini_batch2, shuffle = True)
# test_loader = DataLoader(test_ds, batch_size=50, collate_fn=create_mini_batch2, shuffle = True)

from torch.utils.data import random_split

# Split the training dataset into training and validation sets


# test_size = int(0.5 * len(restaurants_test_ds))
# val_size = len(restaurants_test_ds) - test_size
# test_ds, validation_ds = random_split(restaurants_test_ds, [test_size, val_size])

train_ds = restaurants_train_ds
test_ds = restaurants_test_ds
validation_ds = restaurants_val_ds

# Create DataLoaders
train_loader = DataLoader(train_ds, batch_size=16, collate_fn=create_mini_batch2, shuffle=True)
validation_loader = DataLoader(validation_ds, batch_size=32, collate_fn=create_mini_batch2, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=50, collate_fn=create_mini_batch2, shuffle=True)


In [11]:
# for batch in train_loader:
#     w,x,y,z = batch
#     print(w)
#     print(w.size())
#     print(x)
#     print(x.size())
#     print(y)
#     print(y.size())
#     print(z)
#     print(z.size())
#     break

In [21]:

from torch.optim.lr_scheduler import StepLR
# def train_model_ABSA(loader, epochs):
#     all_data = len(loader)
#     for epoch in range(epochs):
#         finish_data = 0
#         losses = []
#         current_times = []
#         correct_predictions = 0
        
#         for data in loader:
#             t0 = time.time()
#             ids_tensors, segments_tensors, masks_tensors, label_ids = data
#             ids_tensors = ids_tensors.to(DEVICE)
#             segments_tensors = segments_tensors.to(DEVICE)
#             label_ids = label_ids.to(DEVICE)
#             masks_tensors = masks_tensors.to(DEVICE)

#             loss = model_ABSA(ids_tensors=ids_tensors, lable_tensors=label_ids, masks_tensors=masks_tensors, segments_tensors=segments_tensors)
#             losses.append(loss.item())
#             loss.backward()
#             optimizer_ABSA.step()
#             optimizer_ABSA.zero_grad()

#             finish_data += 1
#             current_times.append(round(time.time()-t0,3))
#             current = np.mean(current_times)
#             hr, min, sec = evl_time(current*(all_data-finish_data) + current*all_data*(epochs-epoch-1))
#             print('epoch:', epoch, " batch:", finish_data, "/" , all_data, " loss:", np.mean(losses), " hr:", hr, " min:", min," sec:", sec)         

#         save_model(model_ABSA, 'bert_ABSA3.pkl')
#############################################################################
from torch.optim.lr_scheduler import StepLR
from transformers import get_linear_schedule_with_warmup

def validate_model_ABSA(loader):
    model_ABSA.eval()  # Set the model to evaluation mode
    total_loss = 0
    with torch.no_grad():
        for data in loader:
            ids_tensors, segments_tensors, masks_tensors, label_ids = data
            ids_tensors = ids_tensors.to(DEVICE)
            segments_tensors = segments_tensors.to(DEVICE)
            label_ids = label_ids.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            # Compute the loss during validation
            loss = model_ABSA(ids_tensors=ids_tensors, lable_tensors=label_ids, masks_tensors=masks_tensors, segments_tensors=segments_tensors)
            total_loss += loss.item()

    # Return the average validation loss
    return total_loss / len(loader)


def train_model_ABSA(loader, validation_loader, epochs):

    total_steps = len(loader) * epochs
    warmup_steps = int(0.1 * total_steps)  # 10% warmup

    # Create the scheduler
    scheduler = get_linear_schedule_with_warmup(
        optimizer_ABSA,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps
    )

    all_data = len(loader)
    best_loss = float('inf')
    patience = 10
    early_stop_counter = 0

    # Scheduler
    #scheduler = StepLR(optimizer_ABSA, step_size=1, gamma=0.9)

    for epoch in range(epochs):
        finish_data = 0
        losses = []
        current_times = []
        
        model_ABSA.train()  # Ensure model is in training mode
        for data in loader:
            t0 = time.time()
            ids_tensors, segments_tensors, masks_tensors, label_ids = data
            ids_tensors = ids_tensors.to(DEVICE)
            segments_tensors = segments_tensors.to(DEVICE)
            label_ids = label_ids.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            loss = model_ABSA(ids_tensors=ids_tensors, lable_tensors=label_ids, masks_tensors=masks_tensors, segments_tensors=segments_tensors)
            losses.append(loss.item())
            loss.backward()

            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model_ABSA.parameters(), max_norm=1.0)

            optimizer_ABSA.step()

            scheduler.step() #get_linear_schedule_with_warmup

            optimizer_ABSA.zero_grad()

            finish_data += 1
            current_times.append(round(time.time() - t0, 3))
            current = np.mean(current_times)
            hr, min, sec = evl_time(current * (all_data - finish_data) + current * all_data * (epochs - epoch - 1))
            print('epoch:', epoch, " batch:", finish_data, "/", all_data, " loss:", np.mean(losses), " hr:", hr, " min:", min, " sec:", sec)

            # Periodic validation within the epoch
            if finish_data % 100 == 0:
                val_loss = validate_model_ABSA(validation_loader)
                print(f"Validation Loss after {finish_data} batches: {val_loss}")

        # Print learning rate
        for param_group in optimizer_ABSA.param_groups:
            print(f"Learning rate: {param_group['lr']}")

        # Evaluate on validation set
        val_loss = validate_model_ABSA(validation_loader)
        print(val_loss)
        if val_loss < best_loss:
            best_loss = val_loss
            early_stop_counter = 0
            save_model(model_ABSA, 'bert_ABSA6.pkl')
            print("Saving...")
        else:
            early_stop_counter += 1
            if early_stop_counter >= patience:
                print("Early stopping!")
                break

        # Step scheduler
        #sscheduler.step()



def test_model_ABSA(loader):
    pred = []
    trueth = []
    with torch.no_grad():
        for data in loader:

            ids_tensors, segments_tensors, masks_tensors, label_ids = data
            ids_tensors = ids_tensors.to(DEVICE)
            segments_tensors = segments_tensors.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            outputs = model_ABSA(ids_tensors, None, masks_tensors=masks_tensors, segments_tensors=segments_tensors)
            
            _, predictions = torch.max(outputs, dim=1)

            pred += list([int(i) for i in predictions])
            trueth += list([int(i) for i in label_ids])

    return trueth, pred



In [12]:
%time train_model_ABSA(train_loader, validation_loader, epochs=15)

epoch: 0  batch: 1 / 122  loss: 1.2386082410812378  hr: 5  min: 1  sec: 43
epoch: 0  batch: 2 / 122  loss: 1.3027727603912354  hr: 4  min: 2  sec: 30
epoch: 0  batch: 3 / 122  loss: 1.2826074361801147  hr: 3  min: 43  sec: 47
epoch: 0  batch: 4 / 122  loss: 1.2728183567523956  hr: 3  min: 43  sec: 9
epoch: 0  batch: 5 / 122  loss: 1.2616460561752318  hr: 3  min: 52  sec: 24
epoch: 0  batch: 6 / 122  loss: 1.2394850850105286  hr: 3  min: 41  sec: 40
epoch: 0  batch: 7 / 122  loss: 1.2411294834954398  hr: 3  min: 37  sec: 41
epoch: 0  batch: 8 / 122  loss: 1.2542251497507095  hr: 3  min: 34  sec: 51
epoch: 0  batch: 9 / 122  loss: 1.2320403258005779  hr: 3  min: 27  sec: 31
epoch: 0  batch: 10 / 122  loss: 1.2267006158828735  hr: 3  min: 29  sec: 33
epoch: 0  batch: 11 / 122  loss: 1.2286709980531172  hr: 3  min: 32  sec: 28
epoch: 0  batch: 12 / 122  loss: 1.222141255935033  hr: 3  min: 33  sec: 42
epoch: 0  batch: 13 / 122  loss: 1.2278896386806781  hr: 3  min: 32  sec: 0
epoch: 0  bat

In [19]:
model_ABSA = load_model(model_ABSA, 'bert_ABSA6.pkl')

C:\Users\emink\AppData\Local\Temp\ipykernel_8704\1419488891.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path), strict=False)


In [22]:
%time x, y = test_model_ABSA(test_loader)
print(classification_report(x, y, target_names=[str(i) for i in range(3)]))

CPU times: total: 2min 50s
Wall time: 1min 21s
              precision    recall  f1-score   support

           0       0.69      0.89      0.77       196
           1       0.64      0.43      0.52       196
           2       0.78      0.79      0.78       227

    accuracy                           0.71       619
   macro avg       0.70      0.70      0.69       619
weighted avg       0.70      0.71      0.70       619



# ATE + ABSA

In [9]:
def predict_model_ABSA(sentence, aspect, tokenizer):
    t1 = tokenizer.tokenize(sentence)
    t2 = tokenizer.tokenize(aspect)

    word_pieces = ['[cls]']
    word_pieces += t1
    word_pieces += ['[sep]']
    word_pieces += t2

    segment_tensor = [0] + [0]*len(t1) + [0] + [1]*len(t2)

    ids = tokenizer.convert_tokens_to_ids(word_pieces)
    input_tensor = torch.tensor([ids]).to(DEVICE)
    segment_tensor = torch.tensor(segment_tensor).to(DEVICE)

    print(f"Tokens: {word_pieces}")
    print(f"Segment Tensor: {segment_tensor}")

    with torch.no_grad():
        outputs = model_ABSA(input_tensor, None, None, segments_tensors=segment_tensor)
        _, predictions = torch.max(outputs, dim=1)
    
    return word_pieces, predictions, outputs

def predict_model_ATE(sentence, tokenizer):
    word_pieces = []
    tokens = tokenizer.tokenize(sentence)
    word_pieces += tokens

    ids = tokenizer.convert_tokens_to_ids(word_pieces)
    input_tensor = torch.tensor([ids]).to(DEVICE)

    with torch.no_grad():
        outputs = model_ATE(input_tensor, None, None)
        _, predictions = torch.max(outputs, dim=2)
    predictions = predictions[0].tolist()

    return word_pieces, predictions, outputs

def ATE_ABSA(text):
    terms = []
    word = ""
    
    # ATE
    x, y, _ = predict_model_ATE(text, tokenizer)
    for i in range(len(y)):
        if y[i] == 1:  # Start of a new term
            if len(word) != 0:
                terms.append(word.replace(" ##", ""))
            word = x[i]
        elif y[i] == 2:  # Continuation of the term
            if x[i].startswith("##"):
                word += x[i][2:]  # Remove the '##' prefix
            else:
                word += " " + x[i]
        else:
            if len(word) != 0:
                terms.append(word.replace(" ##", ""))
                word = ""
    if len(word) != 0:
        terms.append(word.replace(" ##", ""))  # Add the last term
    
    print("tokens:", x)
    print("ATE:", terms)

    # ABSA
    if len(terms) != 0:
        for term in terms:
            _, sentiment_class, sentiment_logits = predict_model_ABSA(text, term, tokenizer)

####################################################################################
# def ATE_ABSA(text):
#     terms = []
#     current_term = ""
    
#     # ATE: Aspect Term Extraction
#     tokens, predictions, _ = predict_model_ATE(text, tokenizer)
#     for token, tag in zip(tokens, predictions):
#         if tag == 1:  # Start of a new term
#             if current_term:  # Append the previous term if it exists
#                 terms.append(current_term.strip())  # Finalize the previous term
#             current_term = token  # Start a new term
#         elif tag == 2:  # Continuation of the term
#             if current_term:
#                 if token.startswith("##"):
#                     current_term += token[2:]  # Append subword without '##'
#                 else:
#                     current_term += " " + token
#             else:
#                 # Handle unexpected cases where continuation appears without a start
#                 current_term = token
#         else:  # Tag is 0 (outside)
#             if current_term:  # Append the previous term if it exists
#                 terms.append(current_term.strip())
#                 current_term = ""
#     if current_term:
#         terms.append(current_term.strip())  # Add the last term

#     # **Handle Incorrect Subword Splits**
#     terms = ["".join(term.split(" ##")) for term in terms]

#     print("Tokens:", tokens)
#     print("Extracted Aspect Terms (ATE):", terms)

#     # ABSA: Aspect-Based Sentiment Analysis
#     results = []
#     if terms:
#         for term in terms:
#             _, sentiment_class, sentiment_logits = predict_model_ABSA(text, term, tokenizer)
#             results.append({
#                 "aspect_term": term,
#                 "sentiment_class": sentiment_class.item(),
#                 "logits": sentiment_logits.tolist()
#             })
    
#     # Display Results
#     print("\nAspect Terms and Sentiments (ABSA):")
#     for res in results:
#         print(f"Aspect: {res['aspect_term']}, Sentiment Class: {res['sentiment_class']}, Logits: {res['logits']}")

#     return results



In [10]:
model_ABSA = load_model(model_ABSA, 'bert_ABSA6.pkl')
model_ATE = load_model(model_ATE, 'bert_ATE.pkl')

C:\Users\emink\AppData\Local\Temp\ipykernel_8704\1419488891.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path), strict=False)


In [11]:
text = "the service was discusting."
ATE_ABSA(text)

tokens: ['the', 'service', 'was', 'discus', '##ting', '.']
ATE: ['service']
Tokens: ['[cls]', 'the', 'service', 'was', 'discus', '##ting', '.', '[sep]', 'service']
Segment Tensor: tensor([0, 0, 0, 0, 0, 0, 0, 0, 1])


In [12]:
text = " food was super amazing."
ATE_ABSA(text)

tokens: ['food', 'was', 'super', 'amazing', '.']
ATE: ['food']
Tokens: ['[cls]', 'food', 'was', 'super', 'amazing', '.', '[sep]', 'food']
Segment Tensor: tensor([0, 0, 0, 0, 0, 0, 0, 1])


In [13]:
text = "The pasta was delicious, but the service was slow."
ATE_ABSA(text)

tokens: ['the', 'pasta', 'was', 'delicious', ',', 'but', 'the', 'service', 'was', 'slow', '.']
ATE: ['pasta', 'service']
Tokens: ['[cls]', 'the', 'pasta', 'was', 'delicious', ',', 'but', 'the', 'service', 'was', 'slow', '.', '[sep]', 'pasta']
Segment Tensor: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
Tokens: ['[cls]', 'the', 'pasta', 'was', 'delicious', ',', 'but', 'the', 'service', 'was', 'slow', '.', '[sep]', 'service']
Segment Tensor: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])


In [23]:
text = "The ambiance was fantastic, but the food was overpriced."
ATE_ABSA(text)

tokens: ['the', 'am', '##bian', '##ce', 'was', 'fantastic', ',', 'but', 'the', 'food', 'was', 'over', '##pr', '##ice', '##d', '.']
ATE: ['am', '##bian', '##ce', 'food']
Tokens: ['[cls]', 'the', 'am', '##bian', '##ce', 'was', 'fantastic', ',', 'but', 'the', 'food', 'was', 'over', '##pr', '##ice', '##d', '.', '[sep]', 'am']
Segment Tensor: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
Tokens: ['[cls]', 'the', 'am', '##bian', '##ce', 'was', 'fantastic', ',', 'but', 'the', 'food', 'was', 'over', '##pr', '##ice', '##d', '.', '[sep]', '#', '#', 'bi', '##an']
Segment Tensor: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1])
Tokens: ['[cls]', 'the', 'am', '##bian', '##ce', 'was', 'fantastic', ',', 'but', 'the', 'food', 'was', 'over', '##pr', '##ice', '##d', '.', '[sep]', '#', '#', 'ce']
Segment Tensor: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1])
Tokens: ['[cls]', 'the', 'am', '##bian', '##ce', 'was', 'fantastic', ',', 'bu

In [25]:
text="The waiter was very friendly, and the desserts were outstanding"
ATE_ABSA(text)

tokens: ['the', 'waiter', 'was', 'very', 'friendly', ',', 'and', 'the', 'dessert', '##s', 'were', 'outstanding']
ATE: ['waiter', 'dessert', '##s']
Tokens: ['[cls]', 'the', 'waiter', 'was', 'very', 'friendly', ',', 'and', 'the', 'dessert', '##s', 'were', 'outstanding', '[sep]', 'waiter']
Segment Tensor: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
Tokens: ['[cls]', 'the', 'waiter', 'was', 'very', 'friendly', ',', 'and', 'the', 'dessert', '##s', 'were', 'outstanding', '[sep]', 'dessert']
Segment Tensor: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
Tokens: ['[cls]', 'the', 'waiter', 'was', 'very', 'friendly', ',', 'and', 'the', 'dessert', '##s', 'were', 'outstanding', '[sep]', '#', '#', 's']
Segment Tensor: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1])
